In [1]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = glob.glob('../features/*.pkl')
for l in fl:
    print("'{}'".format(l))

'../features/vgg16_1_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/resnet_aug1_feat.pkl'
'../features/incept_aug1_feat.pkl'
'../features/other_feat.pkl'
'../features/incept_aug2_feat.pkl'
'../features/cnn_1_aug1_feat.pkl'
'../features/resnet_1_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_5_aug1_feat.pkl'
'../features/incept_1_feat.pkl'
'../features/resnet_aug3_feat.pkl'
'../features/incept_aug3_feat.pkl'
'../features/vgg_aug1_feat.pkl'
'../features/resnet_aug2_feat.pkl'


In [10]:
choose_fl = [
'../features/other_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/vgg_aug1_feat.pkl',
'../features/incept_aug2_feat.pkl',
'../features/resnet_aug3_feat.pkl',
    
]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/other_feat.pkl
../features/cnn_1_aug1_feat.pkl
../features/cnn_2_aug1_feat.pkl
../features/cnn_3_aug1_feat.pkl
../features/cnn_4_aug1_feat.pkl
../features/cnn_4_aug1_feat_add_early.pkl
../features/cnn_5_aug1_feat.pkl
../features/vgg_aug1_feat.pkl
../features/incept_aug2_feat.pkl
../features/resnet_aug3_feat.pkl
(1604, 20) (8424, 20) (1604,)


In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=100)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [13]:
cv_test(5,42,True)

[0]	train-logloss:0.639763	valid-logloss:0.643381
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.094194	valid-logloss:0.17904
Stopping. Best iteration:
[101]	train-logloss:0.09382	valid-logloss:0.178767

[0]	train-logloss:0.640312	valid-logloss:0.641876
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.098177	valid-logloss:0.158231
Stopping. Best iteration:
[126]	train-logloss:0.084901	valid-logloss:0.157481

[0]	train-logloss:0.640328	valid-logloss:0.642502
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.096753	valid-logloss:0.183324
Stopping. Best iteration:
[118]	train-logloss:0.088123	valid-logloss:0.181768

[0]	tr

In [12]:
cv_test(5,233,True)

[0]	train-logloss:0.640909	valid-logloss:0.640024
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.099771	valid-logloss:0.152641
Stopping. Best iteration:
[110]	train-logloss:0.093476	valid-logloss:0.150591

[0]	train-logloss:0.640652	valid-logloss:0.638698
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.109804	valid-logloss:0.113212
[200]	train-logloss:0.065203	valid-logloss:0.095043
[300]	train-logloss:0.041004	valid-logloss:0.090215
[400]	train-logloss:0.027138	valid-logloss:0.08842
Stopping. Best iteration:
[466]	train-logloss:0.021269	valid-logloss:0.085581

[0]	train-logloss:0.639568	valid-logloss:0.642072
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't imp